In [29]:
import duckdb
from tabulate import tabulate
from pyspark.sql import SparkSession

In [2]:
conn = duckdb.connect(database=':memory:')

# Create example tables
conn.execute("""
CREATE TABLE employees_2023 (employee_id INTEGER, employee_name VARCHAR);
CREATE TABLE employees_2024 (employee_id INTEGER, employee_name VARCHAR);
""")

# Insert sample data
conn.execute("""
INSERT INTO employees_2023 VALUES
(1, 'Alice'), (2, 'Bob'), (3, 'Charlie');
INSERT INTO employees_2024 VALUES
(1, 'Alice'), (4, 'David'), (5, 'Eve');
""")

In [22]:
select_employees_2023_query = """
SELECT employee_id, employee_name FROM employees_2023;
"""
union_result = conn.execute(select_employees_2023_query).fetchdf()
print(tabulate(union_result, headers = 'keys', tablefmt = 'pretty'))

+---+-------------+---------------+
|   | employee_id | employee_name |
+---+-------------+---------------+
| 0 |      1      |     Alice     |
| 1 |      2      |      Bob      |
| 2 |      3      |    Charlie    |
+---+-------------+---------------+


In [23]:
select_employees_2024_query = """
SELECT employee_id, employee_name FROM employees_2024;
"""
union_result = conn.execute(select_employees_2024_query).fetchdf()
print(tabulate(union_result, headers = 'keys', tablefmt = 'pretty'))

+---+-------------+---------------+
|   | employee_id | employee_name |
+---+-------------+---------------+
| 0 |      1      |     Alice     |
| 1 |      4      |     David     |
| 2 |      5      |      Eve      |
+---+-------------+---------------+


In [28]:
union_query = """
SELECT * FROM employees_2023
UNION
SELECT * FROM employees_2024;
"""
union_result = conn.execute(union_query).fetchdf()
print(tabulate(union_result, headers = 'keys', tablefmt = 'pretty'))

+---+-------------+---------------+
|   | employee_id | employee_name |
+---+-------------+---------------+
| 0 |      4      |     David     |
| 1 |      5      |      Eve      |
| 2 |      2      |      Bob      |
| 3 |      1      |     Alice     |
| 4 |      3      |    Charlie    |
+---+-------------+---------------+


In [25]:
union_all_query = """
SELECT employee_id, employee_name FROM employees_2023
UNION ALL
SELECT employee_id, employee_name FROM employees_2024;
"""
union_all_result = conn.execute(union_all_query).fetchdf()
print(tabulate(union_all_result, headers = 'keys', tablefmt = 'pretty'))

+---+-------------+---------------+
|   | employee_id | employee_name |
+---+-------------+---------------+
| 0 |      1      |     Alice     |
| 1 |      2      |      Bob      |
| 2 |      3      |    Charlie    |
| 3 |      1      |     Alice     |
| 4 |      4      |     David     |
| 5 |      5      |      Eve      |
+---+-------------+---------------+


In [26]:
explain_union_query = """
EXPLAIN ANALYZE
SELECT employee_id, employee_name FROM employees_2023
UNION
SELECT employee_id, employee_name FROM employees_2024;
"""
explain_union_result = conn.execute(explain_union_query).fetchdf()
print(tabulate(explain_union_result, headers = 'keys', tablefmt = 'pretty'))

+---+---------------+--------------------------------------------------------------------------------------------------------------------------------------+
|   |  explain_key  |                                                            explain_value                                                             |
+---+---------------+--------------------------------------------------------------------------------------------------------------------------------------+
| 0 | analyzed_plan |                                               ┌─────────────────────────────────────┐                                                |
|   |               |                                               │┌───────────────────────────────────┐│                                                |
|   |               |                                               ││    Query Profiling Information    ││                                                |
|   |               |                                     

In [27]:
explain_union_all_query = """
EXPLAIN ANALYZE
SELECT employee_id, employee_name FROM employees_2023
UNION ALL
SELECT employee_id, employee_name FROM employees_2024;
"""
explain_union_all_result = conn.execute(explain_union_all_query).fetchdf()
print(tabulate(explain_union_all_result, headers = 'keys', tablefmt = 'pretty'))

+---+---------------+------------------------------------------------------------------------------------------------------------------------------------------+
|   |  explain_key  |                                                              explain_value                                                               |
+---+---------------+------------------------------------------------------------------------------------------------------------------------------------------+
| 0 | analyzed_plan |                                                 ┌─────────────────────────────────────┐                                                  |
|   |               |                                                 │┌───────────────────────────────────┐│                                                  |
|   |               |                                                 ││    Query Profiling Information    ││                                                  |
|   |               |             

In [30]:
# Initialize Spark session
spark = SparkSession.builder.appName("example").getOrCreate()

# Create DataFrames
employees_2023 = spark.createDataFrame([(1, "Alice"), (2, "Bob")], ["id", "name"])
employees_2024 = spark.createDataFrame([(1, "Alice"), (3, "Charlie")], ["id", "name"])

24/06/24 22:14:16 WARN Utils: Your hostname, paulo resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/06/24 22:14:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/24 22:14:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/24 22:14:23 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


24/06/25 04:42:32 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 21736854 ms exceeds timeout 120000 ms
24/06/25 04:42:37 WARN SparkContext: Killing executors is not supported by current scheduler.
24/06/25 04:42:39 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint

In [ ]:
union_all_df = employees_2023.union(employees_2024)

In [ ]:
union_df = employees_2023.union(employees_2024).distinct()